In [1]:
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import random
import argparse
import networkx as nx
from itertools import combinations
import time

In [2]:
def read_dimacs(filename): # int num_vars, array clauses
    clauses = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('p'):
                parts = line.strip().split()
                if len(parts) >= 4:
                    _, _, variables, _ = parts[:4]
                    num_vars = int(variables)
                else:
                    raise ValueError("Invalid CNF file format in 'p' line.")
            elif line.startswith('c') or line.startswith('%') or line.startswith('0'):
                continue
            else:
                clause = list(map(int, line.strip().split()))
                # Remove the trailing 0 if present
                if clause and clause[-1] == 0:
                    clause = clause[:-1]
                if clause:
                    clauses.append(clause)
    return num_vars, clauses

Test for today: if I disable all the heuristics in this particular walksat implementation does it equate the regular one?

if (heuristic3 == walksat):
    plot_intermediate_color_Results();
    plot_intermediate_Statistics(tts99 + cactus plots)

In [3]:

def evaluate_clause(clause, assignment):
    return any(
        (var > 0 and assignment.get(abs(var), False)) or
        (var < 0 and not assignment.get(abs(var), False))
        for var in clause
    )

def get_unsatisfied_clauses(clauses, assignment):
    return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

def get_variables(clauses):
    return set(abs(var) for clause in clauses for var in clause)

def flip_variable(assignment, var):
    assignment[var] = not assignment[var]

def GenerateColors(clauses):
    variables = set(abs(literal) for clause in clauses for literal in clause)
    G = nx.Graph()
    G.add_nodes_from(variables)  # Variable adjacency graph

    # If variables appear in the same clause, make an edge
    for clause in clauses:
        vars_in_clause = set(abs(literal) for literal in clause)
        for var1, var2 in combinations(vars_in_clause, 2):
            G.add_edge(var1, var2)

    # Greedy coloring , return dictionary of unique colors (0 to n) for each node
    colors = nx.coloring.greedy_color(G, strategy='largest_first')
    return colors

def AlgorithmA1(clauses, colors, max_tries, max_flips, p, heuristic_mode=0):
    """    
    clauses: array of clauses from read_cnf() 
    colors: dictionary of color memberships from GenerateColors()
    max_tries: integer of max restarts for algorithm
    max_loops: integer of iterations of algorithm for a given try
    p: probability of greedy vs random selection
    heuristic_mode: 
        0 = greedy in colors from candidate variables to flip
        1 = random from candidate variables to flip
        2 = pick a random color from candidate variables to flip
        3 = always pick first candidate variable in candidate variables to flip
    """
    flips = 0
    variables = np.array(sorted(get_variables(clauses))) # Sorted list of variables
    # 1-based indexing, because 0 in cnf files is used for line breaks
    num_vars = variables[-1]
    color_array = np.zeros(num_vars + 1, dtype=int)
    for var, color in colors.items():
        color_array[var] = color

    # Get the number of unique colors
    unique_colors = np.unique(color_array[color_array > 0])
    
    for _try in range(max_tries):
        # 1) Random assignment
        assignment = np.random.choice([True, False], size=num_vars + 1)  # 1 based indexing
        # Changed it to a vectorized version instead of hash maps to make faster tts

        # 11/11/2024 2:32pm
        # Loops != Tries
        # In regular walksat, we report tries, the general idea here was to
        # increase the parallelism of the algorithm by splitting uncorrelated flips
        # but by adding this "loops" parameter, I underreport the number of tries
        # since there are many loops that occur under a given 'try'.

        for _loop in range(max_flips):
            # 2) Gather UNSAT clauses
            unsat_indices = []
            for idx, clause in enumerate(clauses):
                literals = np.array(clause, dtype=int)
                vars_in_clause = np.abs(literals).astype(int)
                signs = literals > 0
                clause_values = assignment[vars_in_clause]  # Get assignment values
                clause_evaluation = np.where(signs, clause_values, ~clause_values)
                # If none evaluate to 1, add to unsat index
                if not np.any(clause_evaluation):
                    unsat_indices.append(idx)

            if not unsat_indices:
                return assignment[1:], _try, _loop, flips  # Success, return

            # 3) From the UNSAT clauses, pick candidate clauses based on how many colors
            random_samples_count = min(len(unique_colors), len(unsat_indices))
            # Ensure we don't sample more clauses than there are UNSAT clauses
            # this was a bug in the other implementation as well.
            selected_indices = np.random.choice(unsat_indices, size=random_samples_count, replace=False)
            selected_clauses = [clauses[i] for i in selected_indices]

            cc_candidates_to_flip = []
            for clause in selected_clauses:
                variables_in_clause = np.abs(np.array(clause, dtype=int))
                
                # Pick a random variable of the clause for a candidate variable 
                if random.random() < p:
                    # Randomly pick a variable from the clause
                    x = np.random.choice(variables_in_clause)
                    cc_candidates_to_flip.append((x, color_array[x]))
                
                # Or pick variable with least break-count
                else:
                    break_counts = []
                    # Break-count is the number of clauses that become unsatisfied when flipping x
                    for x in variables_in_clause:
                        # Flip variable
                        assignment[x] = ~assignment[x]
                        # Evaluate the number of clauses that become unsatisfied
                        num_new_unsat = 0
                        # Same as before...
                        for clause_check in clauses: 
                            literals_check = np.array(clause_check, dtype=int)
                            vars_in_clause_check = np.abs(literals_check).astype(int)
                            signs_check = literals_check > 0
                            clause_values_check = assignment[vars_in_clause_check]
                            clause_evaluation_check = np.where(signs_check, clause_values_check, ~clause_values_check)
                            if not np.any(clause_evaluation_check):
                                num_new_unsat += 1

                        break_counts.append(num_new_unsat)
                        # Flip variable back
                        assignment[x] = ~assignment[x]

                    # Find variables with least break count
                    min_break = np.min(break_counts)
                    min_indices = np.where(break_counts == min_break)[0]
                    
                    # Pick one index at random if there is more than one this is the tiebreaker
                    idx_min = np.random.choice(min_indices)
                    x = variables_in_clause[idx_min]
                    cc_candidates_to_flip.append((x, color_array[x]))

            # 4) Gather all the picked variables into the candidate list of variables
            color_to_candidates = {}
            for x, color in cc_candidates_to_flip:
                color_to_candidates.setdefault(color, []).append(x)

            # 5) Heuristically pick which variables to flip:
            if heuristic_mode == 0:
                # 5a) Flip variables of the color with the largest number of variables
                selected_color = max(color_to_candidates.keys(), key=lambda c: len(color_to_candidates[c]))
                candidates_in_color = color_to_candidates[selected_color]
                assignment[candidates_in_color] = ~assignment[candidates_in_color]
                flips += len(candidates_in_color)
            elif heuristic_mode == 1:
                # 5b) Randomly pick a variable from candidate variables to flip
                var_to_flip = np.random.choice([x for x, _ in cc_candidates_to_flip])
                assignment[var_to_flip] = ~assignment[var_to_flip]
                flips += 1
            elif heuristic_mode == 2:
                # 5c) Randomly pick a color, flip all variables of that color
                selected_color = np.random.choice(list(color_to_candidates.keys()))
                candidates_in_color = color_to_candidates[selected_color]
                assignment[candidates_in_color] = ~assignment[candidates_in_color]
                flips += len(candidates_in_color)
            elif heuristic_mode == 3:
                # 5d) Always pick the first candidate variable to flip
                var_to_flip = cc_candidates_to_flip[0][0]
                assignment[var_to_flip] = ~assignment[var_to_flip]
                flips += 1

    return "FAIL"

In [4]:
import os
import glob
import time
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np

def extract_problem_number(filename) -> int:
    """Extract numerical problem number from filename."""
    basename = os.path.basename(filename)
    num_str = ''.join(filter(str.isdigit, basename)) # jank the digits
    return int(num_str)

def process_file(cnf_file):
    import time  # need if run in a separate process

    max_flips_var = 10000

    problem_name = os.path.basename(cnf_file)
    num_vars, clauses = read_dimacs(cnf_file)
    start = time.perf_counter()

    colors = GenerateColors(clauses)
    p = 0.5
    solution_object = AlgorithmA1(clauses, colors, max_tries=1, max_flips=max_flips_var, p=0.5, heuristic_mode=3)
    end = time.perf_counter()

    if solution_object != "FAIL":  # we found a satisfying model
        time_to_solution = end - start  # seconds
        flips = solution_object[2]
        print(f"{problem_name}: Time = {time_to_solution:.4f}s, Flips = {flips}")
        return (problem_name, time_to_solution, flips)
    else:
        print(f"{problem_name}: Failed to find a solution")
        # Return None for time and flips (thanks George for suggestion)
        return (problem_name, None, None)

def main():
    problem_names = []
    times_to_solution = []
    flips_to_solution = []
    cnf_directory = "/home/dae/SatExperiments/juniper/uf50suiteSATLIB/"
    cnf_files = glob.glob(os.path.join(cnf_directory, "uf50*.cnf")) # get all uf50* files
    cnf_files.sort(key=extract_problem_number) # sort files based on  numerical part of the filename

    total_cpus = multiprocessing.cpu_count()
    num_workers = max(1, int(total_cpus * 0.5)) # use 50% of cores
    print(f"Total CPU cores: {total_cpus}, Using {num_workers} worker processes.")

    with multiprocessing.Pool(processes=num_workers) as pool: # do the benchmarking in parallel
        results = pool.map(process_file, cnf_files)

    if results:
        problem_names, times_to_solution, flips_to_solution = zip(*results)
    else:
        print("No results error.")
        return

    times_to_solution = np.array(times_to_solution, dtype=float)
    flips_to_solution = np.array(flips_to_solution, dtype=float)

    # take only the first 100 results to plot
    times_to_solution = times_to_solution[0:100]
    flips_to_solution = flips_to_solution[0:100]
    problem_names = problem_names[0:100]

    # Time to solution
    plt.figure(figsize=(14, 7))
    plt.plot(problem_names, times_to_solution, marker='o', linestyle='-', label='Time to Solution')
    plt.xlabel('Problem Name')
    plt.ylabel('Time to Solution TTS (s)')
    plt.title('Time to Solution for First 100 Problems in uf50suiteSATLIB for 100000 MaxFlips 1 Try')
    plt.xticks(rotation=90)
    plt.grid(True)

    failed_indices = np.where(np.isnan(times_to_solution))[0] # if there are any NaN solutions (fails) highlight them
    if failed_indices.size > 0:
        plt.scatter(np.array(problem_names)[failed_indices], 
                    np.zeros_like(failed_indices), 
                    color='red', marker='x', label='Failed to find a solution')

    plt.legend()
    plt.tight_layout()
    plt.show()

    # Flips to solution
    plt.figure(figsize=(14, 7))
    plt.plot(problem_names, flips_to_solution, marker='o', linestyle='-', label='Flips to Solution')
    plt.xlabel('Problem Name')
    plt.ylabel('Flips to Solution (int)')
    plt.title('Flips to Solution for First 100 Problems in uf50suiteSATLIB for 100000 MaxFlips 1 Try')
    plt.xticks(rotation=90)
    plt.grid(True)

    if failed_indices.size > 0: # if there are any NaN solutions (fails) highlight them
        plt.scatter(np.array(problem_names)[failed_indices], 
                    np.zeros_like(failed_indices), 
                    color='red', marker='x', label='Failed to find a solution')

    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


Total CPU cores: 64, Using 32 worker processes.
uf50073.cnf: Time = 1.0418s, Flips = 61
uf500105.cnf: Time = 1.1528s, Flips = 77
uf50081.cnf: Time = 1.2439s, Flips = 75
uf500201.cnf: Time = 1.7980s, Flips = 231
uf500185.cnf: Time = 1.9070s, Flips = 119
uf500225.cnf: Time = 2.1336s, Flips = 148
uf500137.cnf: Time = 2.4476s, Flips = 136
uf5009.cnf: Time = 2.5696s, Flips = 187
uf500233.cnf: Time = 2.6410s, Flips = 166
uf50082.cnf: Time = 1.6574s, Flips = 211
uf500169.cnf: Time = 2.9805s, Flips = 329
uf500177.cnf: Time = 3.0043s, Flips = 181
uf500121.cnf: Time = 3.4652s, Flips = 209
uf500217.cnf: Time = 3.6833s, Flips = 257
uf500186.cnf: Time = 1.8196s, Flips = 137
uf500178.cnf: Time = 0.7421s, Flips = 40
uf50083.cnf: Time = 1.1223s, Flips = 127
uf500226.cnf: Time = 2.0914s, Flips = 120
uf500170.cnf: Time = 1.3611s, Flips = 106
uf500234.cnf: Time = 1.8017s, Flips = 127
uf50084.cnf: Time = 0.4359s, Flips = 28
uf500209.cnf: Time = 5.0421s, Flips = 449
uf500129.cnf: Time = 5.1020s, Flips = 28

In [ ]:
import os
import glob
import time
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np

def extract_problem_number(filename) -> int:
    """Extract numerical problem number from filename."""
    basename = os.path.basename(filename)
    num_str = ''.join(filter(str.isdigit, basename)) # jank the digits
    return int(num_str)

def process_file(cnf_file):
    import time  # need if run in a separate process

    max_flips_var = 1000

    problem_name = os.path.basename(cnf_file)
    num_vars, clauses = read_dimacs(cnf_file)
    start = time.perf_counter()

    colors = GenerateColors(clauses)
    p = 0.5
    solution_object = AlgorithmA1(clauses, colors, max_tries=1, max_flips=max_flips_var, p=0.5, heuristic_mode=3)
    end = time.perf_counter()

    if solution_object != "FAIL":  # we found a satisfying model
        time_to_solution = end - start  # seconds
        flips = solution_object[2]
        print(f"{problem_name}: Time = {time_to_solution:.4f}s, Flips = {flips}")
        return (problem_name, time_to_solution, flips)
    else:
        print(f"{problem_name}: Failed to find a solution")
        # Return None for time and flips (thanks George for suggestion)
        return (problem_name, None, None)

def main():
    problem_names = []
    times_to_solution = []
    flips_to_solution = []
    cnf_directory = "/home/dae/SatExperiments/juniper/uf50suiteSATLIB/"
    cnf_files = glob.glob(os.path.join(cnf_directory, "uf50*.cnf")) # get all uf50* files
    cnf_files.sort(key=extract_problem_number) # sort files based on  numerical part of the filename

    total_cpus = multiprocessing.cpu_count()
    num_workers = max(1, int(total_cpus * 0.5)) # use 50% of cores
    print(f"Total CPU cores: {total_cpus}, Using {num_workers} worker processes.")

    with multiprocessing.Pool(processes=num_workers) as pool: # do the benchmarking in parallel
        results = pool.map(process_file, cnf_files)

    if results:
        problem_names, times_to_solution, flips_to_solution = zip(*results)
    else:
        print("No results error.")
        return

    times_to_solution = np.array(times_to_solution, dtype=float)
    flips_to_solution = np.array(flips_to_solution, dtype=float)

    # take only the first 100 results to plot
    times_to_solution = times_to_solution[0:100]
    flips_to_solution = flips_to_solution[0:100]
    problem_names = problem_names[0:100]

    # Time to solution
    plt.figure(figsize=(14, 7))
    plt.plot(problem_names, times_to_solution, marker='o', linestyle='-', label='Time to Solution')
    plt.xlabel('Problem Name')
    plt.ylabel('Time to Solution TTS (s)')
    plt.title('Time to Solution for First 100 Problems in uf50suiteSATLIB for 100000 MaxFlips 1 Try')
    plt.xticks(rotation=90)
    plt.grid(True)

    failed_indices = np.where(np.isnan(times_to_solution))[0] # if there are any NaN solutions (fails) highlight them
    if failed_indices.size > 0:
        plt.scatter(np.array(problem_names)[failed_indices], 
                    np.zeros_like(failed_indices), 
                    color='red', marker='x', label='Failed to find a solution')

    plt.legend()
    plt.tight_layout()
    plt.show()

    # Flips to solution
    plt.figure(figsize=(14, 7))
    plt.plot(problem_names, flips_to_solution, marker='o', linestyle='-', label='Flips to Solution')
    plt.xlabel('Problem Name')
    plt.ylabel('Flips to Solution (int)')
    plt.title('Flips to Solution for First 100 Problems in uf50suiteSATLIB for 100000 MaxFlips 1 Try')
    plt.xticks(rotation=90)
    plt.grid(True)

    if failed_indices.size > 0: # if there are any NaN solutions (fails) highlight them
        plt.scatter(np.array(problem_names)[failed_indices], 
                    np.zeros_like(failed_indices), 
                    color='red', marker='x', label='Failed to find a solution')

    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


In [ ]:
import os
import glob
import time
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np

def extract_problem_number(filename) -> int:
    """Extract numerical problem number from filename."""
    basename = os.path.basename(filename)
    num_str = ''.join(filter(str.isdigit, basename)) # jank the digits
    return int(num_str)

def process_file(cnf_file):
    import time  # need if run in a separate process

    max_flips_var = 100

    problem_name = os.path.basename(cnf_file)
    num_vars, clauses = read_dimacs(cnf_file)
    start = time.perf_counter()

    colors = GenerateColors(clauses)
    p = 0.5
    solution_object = AlgorithmA1(clauses, colors, max_tries=1, max_flips=max_flips_var, p=0.5, heuristic_mode=3)
    end = time.perf_counter()

    if solution_object != "FAIL":  # we found a satisfying model
        time_to_solution = end - start  # seconds
        flips = solution_object[2]
        print(f"{problem_name}: Time = {time_to_solution:.4f}s, Flips = {flips}")
        return (problem_name, time_to_solution, flips)
    else:
        print(f"{problem_name}: Failed to find a solution")
        # Return None for time and flips (thanks George for suggestion)
        return (problem_name, None, None)

def main():
    problem_names = []
    times_to_solution = []
    flips_to_solution = []
    cnf_directory = "/home/dae/SatExperiments/juniper/uf50suiteSATLIB/"
    cnf_files = glob.glob(os.path.join(cnf_directory, "uf50*.cnf")) # get all uf50* files
    cnf_files.sort(key=extract_problem_number) # sort files based on  numerical part of the filename

    total_cpus = multiprocessing.cpu_count()
    num_workers = max(1, int(total_cpus * 0.5)) # use 50% of cores
    print(f"Total CPU cores: {total_cpus}, Using {num_workers} worker processes.")

    with multiprocessing.Pool(processes=num_workers) as pool: # do the benchmarking in parallel
        results = pool.map(process_file, cnf_files)

    if results:
        problem_names, times_to_solution, flips_to_solution = zip(*results)
    else:
        print("No results error.")
        return

    times_to_solution = np.array(times_to_solution, dtype=float)
    flips_to_solution = np.array(flips_to_solution, dtype=float)

    # take only the first 100 results to plot
    times_to_solution = times_to_solution[0:100]
    flips_to_solution = flips_to_solution[0:100]
    problem_names = problem_names[0:100]

    # Time to solution
    plt.figure(figsize=(14, 7))
    plt.plot(problem_names, times_to_solution, marker='o', linestyle='-', label='Time to Solution')
    plt.xlabel('Problem Name')
    plt.ylabel('Time to Solution TTS (s)')
    plt.title('Time to Solution for First 100 Problems in uf50suiteSATLIB for 100000 MaxFlips 1 Try')
    plt.xticks(rotation=90)
    plt.grid(True)

    failed_indices = np.where(np.isnan(times_to_solution))[0] # if there are any NaN solutions (fails) highlight them
    if failed_indices.size > 0:
        plt.scatter(np.array(problem_names)[failed_indices], 
                    np.zeros_like(failed_indices), 
                    color='red', marker='x', label='Failed to find a solution')

    plt.legend()
    plt.tight_layout()
    plt.show()

    # Flips to solution
    plt.figure(figsize=(14, 7))
    plt.plot(problem_names, flips_to_solution, marker='o', linestyle='-', label='Flips to Solution')
    plt.xlabel('Problem Name')
    plt.ylabel('Flips to Solution (int)')
    plt.title('Flips to Solution for First 100 Problems in uf50suiteSATLIB for 100000 MaxFlips 1 Try')
    plt.xticks(rotation=90)
    plt.grid(True)

    if failed_indices.size > 0: # if there are any NaN solutions (fails) highlight them
        plt.scatter(np.array(problem_names)[failed_indices], 
                    np.zeros_like(failed_indices), 
                    color='red', marker='x', label='Failed to find a solution')

    plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()
